Today we'll cover:

1. [Reading and writing text](#Reading-and-writing-text)
2. [Reading from databases](#Reading-from-databases)
3. [Reading from web APIs](#Reading-from-web-APIs)

# Reading and writing text

The 2 main reading & writing functions in pandas we will discuss are:

* `read_csv()` to read comma separated data (we saw this in the last lecture)
* `to_csv()` to write comnna separated data

But before we get into the details of these, let us create some data files first.

In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame([['German', 1777, 1855],
                   ['Swiss', 1707, 1783],
                   ['French', 1736, 1813],
                   ['French', 1749, 1827]],
                  index=['Gauss', 'Euler', 'Lagrange', 'Laplace'],
                  columns=['Nationality', 'Born', 'Died'])
df.index.name = 'Mathematician'
df

,Nationality,Born,Died
Mathematician,,,
Gauss,German,1777,1855
Euler,Swiss,1707,1783
Lagrange,French,1736,1813
Laplace,French,1749,1827


In [3]:
df.to_csv('math.csv')  # export to a csv file

In [4]:
!cat math.csv  # run shell command to examine file

Mathematician,Nationality,Born,Died
Gauss,German,1777,1855
Euler,Swiss,1707,1783
Lagrange,French,1736,1813
Laplace,French,1749,1827


In [5]:
del df  # delete the DataFrame
df = pd.read_csv('math.csv')  # read it back in
df

,Mathematician,Nationality,Born,Died
0,Gauss,German,1777,1855
1,Euler,Swiss,1707,1783
2,Lagrange,French,1736,1813
3,Laplace,French,1749,1827


That didn't quite give our original DataFrame back. We need to tell `read_csv` to use the first column (column 0) as the index.

In [6]:
df = pd.read_csv('math.csv', index_col=0)  # use columns 0 as index
df

,Nationality,Born,Died
Mathematician,,,
Gauss,German,1777,1855
Euler,Swiss,1707,1783
Lagrange,French,1736,1813
Laplace,French,1749,1827


We can also save a DataFrame without the header and index.

In [7]:
df.to_csv('math.csv', index=False, header=False)

In [8]:
!cat math.csv

German,1777,1855
Swiss,1707,1783
French,1736,1813
French,1749,1827


In [9]:
df = pd.read_csv('math.csv', names=['Nationality', 'Born', 'Died'])  # read and supply columns names
df.index = ['Gauss', 'Euler', 'Lagrange', 'Laplace']   # supply the index
df.index.name = 'Mathematicians'
df

,Nationality,Born,Died
Mathematicians,,,
Gauss,German,1777,1855
Euler,Swiss,1707,1783
Lagrange,French,1736,1813
Laplace,French,1749,1827


For large files, it might make sense to read them in chunks.

In [10]:
df.to_csv('math.csv')  # save again for index and header
chunks = pd.read_csv('math.csv', chunksize=1)  # chunksize is in no. of lines
french_count = 0
for piece in chunks:
    french_count += piece['Nationality'][0] == 'French'
print 'Found %d French mathematicians.' % french_count

Found 2 French mathematicians.


In [11]:
import sys
df.to_csv(sys.stdout, sep=':')  # can use a separator other than comma

Mathematicians:Nationality:Born:Died
Gauss:German:1777:1855
Euler:Swiss:1707:1783
Lagrange:French:1736:1813
Laplace:French:1749:1827


# Reading from databases

Often the data you want to work with resides in a [relational database management system](http://en.wikipedia.org/wiki/Relational_database_management_system) (RDBMS). Some common commerical RDBMS implementations are: Oracle Database, Microsoft SQL server, MySQL and IBM DB2. [SQLite](http://www.sqlite.org/) is a freely available lightweight, disk-based (doesn't require a database server) database engine. In python, the package `sqlite3` provides an interface to SQLite.

SQL (Structured Query Language) is a language used to interact with a database. A quick introduction to SQL can be found here:

http://www.w3schools.com/sql/

The SQL command to create a table in a database is `CREATE` (SQL queries are not case-sensitive: `CREATE` is the same as `create`).

In [12]:
import sqlite3

query = """
CREATE TABLE math
(Mathematician VARCHAR(10),
 Nationality VARCHAR(10),
 Born INTEGER,
 Died INTEGER
);"""

Now that we have created our first query, let us connect to a database and execute the query. In SQLite, databases are stored on disk as files. However, we can create an in-memory database denoted by the special name `:memory:`.

Any query that changes the database, need to be committed to ensure that the change is visible to other connections to the database that might be open at the same time.

In [13]:
con = sqlite3.connect(':memory:')  # connect to an in-memory database
con.execute(query)  # execute the query
con.commit()  # commit the change

We now have an empty table in the `:memory:` database. Let us insert some values into it using the SQL command `INSERT`.

In [14]:
con.execute("INSERT INTO math VALUES('Gauss', 'German', 1777, 1855)")  # insert values
con.commit()  # and commit

At this stage, the table has a single row in it. Let us retrieve contents of the table using the SQL command `SELECT`. The result of executing a `SELECT` command is a *cursor*. You can think of it as an iterator.

In [15]:
cursor = con.execute('SELECT * FROM math')
for row in cursor:
    print row

(u'Gauss', u'German', 1777, 1855)


Let us insert some more values in our table. The `executemany` method is useful for executing many commands for the same type. The placeholder `?` gets replaces by the values supplied in the list argument.

In [16]:
values = [('Euler', 'Swiss', 1707, 1783),
          ('Lagrange', 'French', 1736, 1813),
          ('Laplace', 'French', 1749, 1827)]
con.executemany("INSERT INTO math VALUES(?, ?, ?, ?)", values)
con.commit()

We have seen that a cursor can be used an iterators. If you want one row, you can use the `fetchone()` method. If you you want all rows, you can use the `fetchall()` method.

In [17]:
cursor = con.execute('SELECT * FROM math')
print "Fetching one row..."
print cursor.fetchone()
print "Fetching all remaining rows..."
print cursor.fetchall()

Fetching one row...
(u'Gauss', u'German', 1777, 1855)
Fetching all remaining rows...
[(u'Euler', u'Swiss', 1707, 1783), (u'Lagrange', u'French', 1736, 1813), (u'Laplace', u'French', 1749, 1827)]


Pandas provides a convenient way to convert results of SQL queries in DataFrame objects.

In [18]:
import pandas.io.sql as sql
sql.read_sql('SELECT * FROM math', con)

,Mathematician,Nationality,Born,Died
0,Gauss,German,1777,1855
1,Euler,Swiss,1707,1783
2,Lagrange,French,1736,1813
3,Laplace,French,1749,1827


Let us try retrieving only French mathematicians.

In [19]:
sql.read_sql('SELECT * FROM math WHERE Nationality="French"', con)

,Mathematician,Nationality,Born,Died
0,Lagrange,French,1736,1813
1,Laplace,French,1749,1827


# Reading from web APIs

We will use the [Twitter Search API](https://dev.twitter.com/rest/public/search) to search for tweets. The API returns results in the [JSON format](http://en.wikipedia.org/wiki/JSON). You will remember that IPython notebooks (such as this document!) are also encoded in the JSON format.

But before we can build a search query and execute it on twitter, we need to authenticate. We will use Twitter's [Application-only authentication](https://dev.twitter.com/oauth/application-only). The way this works is:

1. Our (Python) application will encodes its *consumer key* and *consumer secret* into a specially encoded set of credentials.
2. Then the application will make a request to exchange these credentials for a *bearer token*.
3. When accessing the API, our application will use the *bearer token* to authenticate.

Let us work on Step 1.

In [20]:
from urllib2 import urlopen, Request  # to create HTTP requests and open URLs
import base64  # for base64 encoding
import json  # for handling the JSON format

consumer_key = 'dNcn9ZjPJ6dSaXJMYnVgna7jg'  # our app's consumer key
consumer_secret = open('consumer_secret', 'r').read().strip()  # read secret (should not be made public) from file

bearer_token = '%s:%s' % (consumer_key, consumer_secret)
encoded_bearer_token = base64.b64encode(bearer_token.encode('ascii'))  # bearer token needs to be base64 encoded
request = Request('https://api.twitter.com/oauth2/token')
request.add_header('Content-Type',
                   'application/x-www-form-urlencoded;charset=UTF-8')
request.add_header('Authorization',
                   'Basic %s' % encoded_bearer_token.decode('utf-8'))
request_data = 'grant_type=client_credentials'.encode('ascii')
request.add_data(request_data)

Now that we have a Request object ready, let us send our request to get a bearer token to Twitter (Step 2).

In [21]:
response = urlopen(request)  # make the request
raw_data = response.read().decode('utf-8')  # read the raw results in JSON format
data = json.loads(raw_data)  # decode JSON into Python data structures
bearer_token = data['access_token']  # extract the token

We can now use the bearer token to search (Step 3). Let us search for tweets containing "data science".

In [22]:
url = 'https://api.twitter.com/1.1/search/tweets.json?q=data%20science'  # search for "data science"
request = Request(url)
request.add_header('Authorization', 'Bearer %s' % bearer_token)  # use the bearer token from Step 2
response = urlopen(request)  # make the request
raw_data = response.read().decode('utf-8')  # results in raw JSON
data = json.loads(raw_data)  # decode JSON into Python data structures

At this point `data` is a dictionary with just two keys: `search_metadata` and `statuses`. The latter has the tweets inside a Python list.

In [23]:
print data.keys()

[u'search_metadata', u'statuses']


In [24]:
import pprint  # import pretty print module
pprint.pprint(data['statuses'][0])  # print the first tweet, it is a Python dict

{u'contributors': None,
 u'coordinates': None,
 u'created_at': u'Mon Oct 12 14:49:05 +0000 2015',
 u'entities': {u'hashtags': [{u'indices': [106, 119],
                              u'text': u'agreenskills'}],
               u'symbols': [],
               u'urls': [{u'display_url': u'kaggle.com/competitions',
                          u'expanded_url': u'https://www.kaggle.com/competitions',
                          u'indices': [38, 61],
                          u'url': u'https://t.co/MmLCB6Cc7A'}],
               u'user_mentions': [{u'id': 292832796,
                                   u'id_str': u'292832796',
                                   u'indices': [94, 105],
                                   u'name': u'Bal\xe1zs K\xe9gl',
                                   u'screen_name': u'balazskegl'}]},
 u'favorite_count': 0,
 u'favorited': False,
 u'geo': None,
 u'id': 653583170017853440,
 u'id_str': u'653583170017853440',
 u'in_reply_to_screen_name': None,
 u'in_reply_to_status_id': Non

In [25]:
# Extract the text and created_at fields and convert in pandas DataFrame
tweets_df = pd.DataFrame(data['statuses'], columns=['created_at', 'text'])
tweets_df

,created_at,text
0,Mon Oct 12 14:49:05 +0000 2015,data science competitions from Kaggle https://...
1,Mon Oct 12 14:48:21 +0000 2015,SAS Academy for Data Science Certification Pro...
2,Mon Oct 12 14:48:13 +0000 2015,"RT @iradche: 90+ Active Blogs on Analytics, Bi..."
3,Mon Oct 12 14:47:25 +0000 2015,ReTw Soccermetric: Red cards and race: Importa...
4,Mon Oct 12 14:46:40 +0000 2015,Evolution of NASA Earth Science Data Systems i...
5,Mon Oct 12 14:46:34 +0000 2015,Data Science Analyst http://t.co/JgSliV9au2 #work
6,Mon Oct 12 14:46:29 +0000 2015,hatebu: Building Data Science Teams - http://t...
7,Mon Oct 12 14:46:04 +0000 2015,"RT @MBNConsilium_RR: MBN Data Science Event, L..."
8,Mon Oct 12 14:46:02 +0000 2015,Looking for a #Year #Round Intern-Product Mana...
9,Mon Oct 12 14:45:19 +0000 2015,Data Science Bootcamps – The Best Courses of 2...


In [26]:
url = 'https://api.twitter.com/1.1/search/tweets.json?q=%23datascience'  # search for the hashtag #datascience
request = Request(url)
request.add_header('Authorization', 'Bearer %s' % bearer_token)  # use the bearer token from Step 2
response = urlopen(request)  # make the request
raw_data = response.read().decode('utf-8')  # results in raw JSON
data = json.loads(raw_data)  # decode JSON into Python data structures
hashtag_tweets_df = pd.DataFrame(data['statuses'], columns=['created_at', 'text'])
hashtag_tweets_df

,created_at,text
0,Mon Oct 12 14:49:23 +0000 2015,RT @dataiku: 9 Steps to Become a Data Scientis...
1,Mon Oct 12 14:49:08 +0000 2015,"#AzureML is one beautiful poster, makes it ver..."
2,Mon Oct 12 14:48:55 +0000 2015,RT 3DBillionaire: Leverage your #Enterprise wi...
3,Mon Oct 12 14:46:21 +0000 2015,The price of the #InternetOfThings - http://t....
4,Mon Oct 12 14:45:15 +0000 2015,RT @ORESYS: Petit-déjeuner à suivre demain en ...
5,Mon Oct 12 14:45:06 +0000 2015,RT @ORESYS: Petit-déjeuner à suivre demain en ...
6,Mon Oct 12 14:44:51 +0000 2015,"#Amazon Announces #InternetOfThings Platform, ..."
7,Mon Oct 12 14:44:01 +0000 2015,RT @moorejh: Here’s how one #healthcare org is...
8,Mon Oct 12 14:43:46 +0000 2015,Analyzing the #InternetOfThings: https://t.co/...
9,Mon Oct 12 14:43:33 +0000 2015,RT @moorejh: Here’s how one #healthcare org is...


In [27]:
# clean up temporary files in the end
!rm math.csv